In [1]:
import pandas as pd

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import random
from transformers import ViTForImageClassification, ViTFeatureExtractor, ViTImageProcessor, ViTModel

/Users/vikashmediboina/mambaforge/envs/printing3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
test_data = pd.read_csv("/Users/vikashmediboina/Downloads/early-detection-of-3d-printing-issues/test.csv")
test_data["img_path"]= ["/Users/vikashmediboina/Downloads/early-detection-of-3d-printing-issues/images/" + each for each in test_data["img_path"]]

In [4]:
test_data.head()

,img_path,printer_id,print_id
0,/Users/vikashmediboina/Downloads/early-detecti...,101,1678578332
1,/Users/vikashmediboina/Downloads/early-detecti...,101,1678578332
2,/Users/vikashmediboina/Downloads/early-detecti...,101,1678578332
3,/Users/vikashmediboina/Downloads/early-detecti...,101,1678578332
4,/Users/vikashmediboina/Downloads/early-detecti...,101,1678578332


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps")
print(device)

mps


In [4]:
import torch

# Define the path to the .pth file
pth_file_path = 'modelvision.pth'

# Load the .pth file
loaded_model = torch.load(pth_file_path)

# Access the model parameters or other saved data
model_state_dict = loaded_model['model_state_dict']
optimizer_state_dict = loaded_model['optimizer_state_dict']
# ... (access other saved data as needed)

KeyError: 'model_state_dict'

In [9]:
import torch
import os

# Set up the MPS environment variables
os.environ['LD_PRELOAD'] = '/usr/local/cuda/extras/CUPTI/lib64/libcupti.so:/usr/local/cuda/targets/x86_64-linux/lib/libnvidia-ml.so'
os.environ['MPS_SERVER'] = 'localhost:53331'
os.environ['MPS_VISIBLE_DEVICES'] = '0'

# Load the PyTorch model with MPS support
loaded_model = torch.jit.load("ViTmodel.pt")

# Run the model on input data
input_data = torch.randn(1, 3, 224, 224)
output = loaded_model(input_data)

print(output)


RuntimeError: supported devices include CPU, CUDA and HPU, however got MPS

In [11]:
loaded_model = torch.load("modelvision.pth",map_location=torch.device('cpu'))
input_data = torch.randn(16,3,224,224).to(device)
# loaded_model.()
# Evaluate the model on input data
trained_model = loaded_model(input_data)

RuntimeError: supported devices include CPU, CUDA and HPU, however got MPS

In [22]:
!pip install torch-mps


ERROR: Could not find a version that satisfies the requirement torch-mps (from versions: none)
ERROR: No matching distribution found for torch-mps


In [13]:
import torch
import os

# Set environment variables for MPS
os.environ['MPS_VERBOSE'] = '1'
os.environ['MPS_MAX_LINES'] = '3'
os.environ['MPS_VISIBLE_DEVICES'] = '0'

# Load the PyTorch model with MPS support
model_script = torch.jit.script(torch.load('ViTmodel.pt',map_location=torch.device('cpu')))

# Convert the model to CPU and evaluate
input_data = torch.randn(16, 3, 224, 224).to('cpu')
trained_model = model_script(input_data)

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__.py", line 14, in forward
    input = torch.to((modality1).forward(input1, ), dtype=6, layout=0, device=torch.device("mps"))
    x = torch.to(torch.flatten(input, 1), dtype=6, layout=0, device=torch.device("mps"))
    _0 = torch.to((head).forward(x, ), dtype=6, layout=0, device=torch.device("mps"))
                   ~~~~~~~~~~~~~ <--- HERE
    return _0
class MLP(Module):
  File "code/__torch__.py", line 27, in forward
    _1 = ops.prim.NumToTensor(torch.size(x, 0))
    input = torch.view(x, [int(_1), -1])
    return (layers).forward(input, )
            ~~~~~~~~~~~~~~~ <--- HERE
  File "code/__torch__/torch/nn/modules/container.py", line 53, in forward
    _1 = getattr(self, "1")
    _0 = getattr(self, "0")
    _14 = (_1).forward((_0).forward(input, ), )
                        ~~~~~~~~~~~ <--- HERE
    _15 = (_4).forwar

In [27]:
import torch
from transformers import ViTModel,ViTConfig

# Load the saved state dictionary
loaded_state_dict = torch.load('ViTmodel.pt', map_location=torch.device('cpu'))



# Convert the loaded model to a state dictionary
loaded_state_dict = loaded_model.state_dict()

# Create a new model instance with the same configuration
config = ViTConfig.from_pretrained('google/vit-base-patch16-224-in21k')
new_model = ViTModel(config)

# Load the state dictionary into the new model instance
new_model.load_state_dict(loaded_state_dict, strict=False)

# Set the model to evaluation mode
new_model.eval()


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation(

In [38]:
from sklearn.metrics import f1_score, accuracy_score

# Define the ViT model and feature extractor
img_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

# Define the device to train the model on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model = trained_model.to(device)

predicted_labels=[] 
submission = {}
# Define the testing loop
with torch.no_grad():
    for image_path in test_data["img_path"]:
        image = Image.open(image_path).convert("RGB")
        processed_img = img_processor(image, return_tensors='pt', data_format='channels_first').to(device)
        outputs = trained_model(**processed_img)
        print(outputs)
        # preds_probs = outputs.prediction.softmax(dim=-1)
        _, preds = torch.softmax(outputs, dim=1)
        print(preds.item())
        predicted_labels.append(preds.item())
        submission[image_path] = preds.item()


BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.2293, -0.1140,  0.0780,  ..., -0.1330, -0.0726, -0.1230],
         [ 0.2210, -0.3239,  0.2153,  ..., -0.1833, -0.0287, -0.0542],
         [ 0.3217, -0.1437,  0.1398,  ..., -0.0003, -0.1282, -0.0058],
         ...,
         [ 0.1537, -0.1601,  0.0014,  ..., -0.0038, -0.2342, -0.0676],
         [ 0.1608, -0.1199, -0.0024,  ...,  0.0550, -0.0499, -0.2255],
         [ 0.2512, -0.0451,  0.1070,  ...,  0.0008, -0.0839, -0.0868]]]), pooler_output=tensor([[ 4.2619e-01, -1.3044e-01, -7.9687e-01, -5.4830e-02, -6.7148e-01,
          3.2121e-02,  4.1386e-01,  1.7094e-01,  1.9688e-01, -3.8563e-01,
          5.3947e-01,  1.5896e-01, -2.0352e-01,  3.7294e-01, -2.7611e-01,
         -3.1651e-01, -3.9442e-01, -4.5078e-01,  3.7674e-01,  5.7867e-01,
          7.8278e-01, -1.3906e-01,  2.2333e-01,  3.2622e-01,  4.9351e-01,
         -3.2776e-01, -4.1125e-02,  5.4864e-02,  1.2116e-01,  1.5514e-01,
         -5.9572e-01,  7.3256e-01, -5.1687e-01,  4.637

TypeError: softmax() received an invalid combination of arguments - got (BaseModelOutputWithPooling, dim=int), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype, *, Tensor out)
 * (Tensor input, name dim, *, torch.dtype dtype)
